In [1]:
#extract the chr2 sequence
def extract_sequence(fasta_file, sequence_id):
    sequences = {}
    current_sequence = ''
    
    # Read the FASTA file
    with open(fasta_file, 'r') as file:
        for line in file:
            line = line.strip()
            
            # Check for header line
            if line.startswith('>'):
                if current_sequence:
                    sequences[header] = current_sequence
                    current_sequence = ''
                header = line[1:]
            else:
                current_sequence += line
    
    # Add the last sequence to the dictionary
    if current_sequence:
        sequences[header] = current_sequence
    
    # Retrieve the desired sequence
    if sequence_id in sequences:
        return sequences[sequence_id]
    else:
        return None
    
def save_to_file(data, file_path):
    with open(file_path, 'w') as file:
        file.write(data)

fasta_file = '/data/repository/organisms/GRCm38_ensembl/genome_fasta/genome.fa'
sequence_id = '2 dna_sm:chromosome chromosome:GRCm38:2:1:182113224:1 REF'

extracted_sequence = extract_sequence(fasta_file, sequence_id)
if extracted_sequence:
    print("found!")
    save_to_file(extracted_sequence,"/data/jenuwein/processing/20230412_Alecia_PoreC/chr2.fa")
    print("wrote!")
else:
    print("Sequence ID not found.")

found!
wrote!


In [7]:
#Inserting the sequence at the expected location, keep size of the line the same
def insert_fasta(original_file, insertion_file, insertion_point, output_file):
    # Open the original file for reading
    with open(original_file, 'r') as original:
        # Save the current file position and initialize the line buffer
        original_pos = 0
        line_buffer = ''
        header_line = ''

        # Read the header line from the original file
        header_line = original.readline()

        # Open the insertion file for reading
        with open(insertion_file, 'r') as insertion:
            # Open the output file for writing
            with open(output_file, 'w') as output:
                # Write the header line to the output file
                output.write(header_line)
                #print(header_line)
                # Read the original file character by character
                while True:
                    # Read a character from the original file
                    char = original.read(1)

                    # Check if the insertion point has been reached
                    if original_pos == insertion_point:
                        # Get the length of the prior line
                        line_length = len(line_buffer.rstrip('\r\n'))
                        print("hello")
                        # Write the insertion file
                        for insertion_line in insertion:
                            # Trim the insertion line to match the line length
                            
                            insertion_line = insertion_line.rstrip('\r\n')[:line_length] + '\n'
                            output.write(insertion_line)
                        
                        # Reset the file pointer of the insertion file to the beginning
                        insertion.seek(0)

                        # Write the remaining content of the original file
                        output.write(char)
                        output.write(original.read())
                        break

                    # Update the line buffer and position
                    line_buffer += char
                    original_pos += 1
                    
                    if original_pos % 100 == 0:
                        print(str(original_pos))

                    # Check if a newline character is encountered
                    if char == '\n':
                        # Reset the line buffer
                        line_buffer = ''

                    # Check if the end of the file is reached
                    if not char:
                        # Set the line length to match the last line
                        line_length = len(line_buffer.rstrip('\r\n'))

                        # Write the insertion file at the end of the file
                        for insertion_line in insertion:
                            # Trim the insertion line to match the line length
                            insertion_line = insertion_line.rstrip('\r\n')[:line_length] + '\n'
                            output.write(insertion_line)
                        break          
                        
original_file = "/data/jenuwein/processing/20230412_Alecia_PoreC/chr2.fa"
insertion_file = '/data/jenuwein/processing/20230412_Alecia_PoreC/insertion.fa'
insertion_point = '116551619'  # Modify with the desired position
output_file = '/data/jenuwein/processing/20230412_Alecia_PoreC/chr2_plusInsertion.fa'

# Call the function to perform the insertion
insert_fasta(original_file, insertion_file, insertion_point, output_file)


In [8]:
def modify_file(filename):
    # Read the contents of the file
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Get the length of the first line
    first_line = lines[0].strip()
    first_line_length = len(first_line)

    # Modify lines starting from the second line
    for i in range(1, len(lines)):
        line = lines[i].strip()
        if len(line) > first_line_length:
            # Trim the line if it's longer than the first line
            line = line[:first_line_length]
        else:
            # Pad the line if it's shorter than the first line
            line = line + ' ' * (first_line_length - len(line))

        # Update the line in the list
        lines[i] = line + '\n'

    # Write the modified lines back to the file
    with open(filename, 'w') as file:
        file.writelines(lines)

# Usage example
modify_file('/data/jenuwein/processing/20230412_Alecia_PoreC/chr2_withInsertion.fa')